In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

In [2]:
client = MongoClient('localhost', 27017)
db = client.TFE
collection = db.results

In [3]:
def generateFromCM(cm):
    tn, fp, fn, tp = cm
    y_true = ['reliable' for i in range(0, tn + fp)]
    y_pred = ['reliable' for i in range(0, tn)]+['fake' for i in range(0, fp)]
    y_true = y_true + ['fake' for i in range(0, fn + tp)]
    y_pred = y_pred + ['reliable' for i in range(0, fn)] + ['fake' for i in range(0, tp)]
    return y_true, y_pred

In [4]:
cursor = collection.find({'model' : 'Attention LSTM 3.1', 'finish' : True, 'params' : {'$exists' : True}}, no_cursor_timeout=True)
pbar = tqdm(total=504)
for i, res in enumerate(cursor):
    for i, result in enumerate(res['result']):
        epoch = result['epoch']
        cm = result['valid']['Confusion Matrix']
        y_true, y_pred = generateFromCM(cm)
        clr = classification_report(y_true, y_pred, output_dict = True)
        collection.update_one({'experiment_id' : res['experiment_id'], 'result.epoch' : epoch}, {"$set" : {'result.$.valid.clr' : clr}})
        
        cm = result['train']['Confusion Matrix']
        y_true, y_pred = generateFromCM(cm)
        clr = classification_report(y_true, y_pred, output_dict = True)
        collection.update_one({'experiment_id' : res['experiment_id'], 'result.epoch' : epoch}, {"$set" : {'result.$.train.clr' : clr}})
    pbar.update(1)
pbar.close()
cursor.close()

  1%|          | 3/504 [00:45<2:06:55, 15.20s/it]/home/simon/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|██████████| 504/504 [2:13:10<00:00, 15.21s/it]


In [5]:
clr

{'fake': {'precision': 0.42140151515151514,
  'recall': 0.1983065953654189,
  'f1-score': 0.2696969696969697,
  'support': 4488},
 'reliable': {'precision': 0.5573326771653543,
  'recall': 0.7875521557719054,
  'f1-score': 0.6527377521613833,
  'support': 5752},
 'accuracy': 0.529296875,
 'macro avg': {'precision': 0.4893670961584347,
  'recall': 0.4929293755686621,
  'f1-score': 0.46121736092917653,
  'support': 10240},
 'weighted avg': {'precision': 0.4977565975639763,
  'recall': 0.529296875,
  'f1-score': 0.484858159221902,
  'support': 10240}}